In [1]:
import pandas as pd

NROWS = 100000
n_input = 50

data = pd.read_csv('hn_title_norm_vec.csv', nrows=NROWS)

In [2]:
clean_data = data.drop(labels=['id'], axis=1)

In [3]:
train = clean_data.sample(frac=0.8)
test = clean_data.drop(train.index)

In [4]:
import numpy as np
train_np = train.as_matrix()
test_np = test.as_matrix()
print(train_np.shape)
print(test_np.shape)

(80000, 1200)
(20000, 1200)


In [16]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 1  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_title = Input(shape=(train_np.shape[1],))
# "encoded" is the encoded representation of the input
#encoded_0 = Dense(1200, activation='tanh')(input_title)
encoded = Dense(encoding_dim, activation='tanh')(input_title)
# "decoded" is the lossy reconstruction of the input
#decoded_0 = Dense(1200, activation='tanh')(encoded)
decoded = Dense(train_np.shape[1], activation='tanh')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input=input_title, output=decoded)

# this model maps an input to its encoded representation
encoder = Model(input=input_title, output=encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(train_np, train_np,
                nb_epoch=50,
                batch_size=1000,
                shuffle=True,
                validation_data=(test_np, test_np))

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 8s - loss: 0.0061 - val_loss: 0.0055
Epoch 2/50
80000/80000 [==============================] - 8s - loss: 0.0052 - val_loss: 0.0051
Epoch 3/50
80000/80000 [==============================] - 8s - loss: 0.0050 - val_loss: 0.0049
Epoch 4/50
80000/80000 [==============================] - 8s - loss: 0.0049 - val_loss: 0.0048
Epoch 5/50
80000/80000 [==============================] - 8s - loss: 0.0048 - val_loss: 0.0048
Epoch 6/50
80000/80000 [==============================] - 8s - loss: 0.0047 - val_loss: 0.0047
Epoch 7/50
80000/80000 [==============================] - 8s - loss: 0.0047 - val_loss: 0.0047
Epoch 8/50
80000/80000 [==============================] - 8s - loss: 0.0047 - val_loss: 0.0047
Epoch 9/50
80000/80000 [==============================] - 8s - loss: 0.0047 - val_loss: 0.0047
Epoch 10/50
80000/80000 [==============================] - 8s - loss: 0.0047 - val_loss: 0.0047

In [56]:
i0 = train_np[0:1]
print('a', i0[0][23*50:24*50], i0.shape)

e0 = encoder.predict(i0)
print(e0, e0.shape)

o0 = decoder.predict(e0)
print('b', o0[0][23*50:24*50], o0.shape)

print(np.sum(np.square(i0[0][23*50:24*50] - o0[0][23*50:24*50])) / 50)

a [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] (1, 1200)
[[-0.34407094]] (1, 1)
b [  5.04688178e-05  -9.16062127e-05  -7.42504126e-05   6.98399817e-05
  -9.92970381e-05  -8.90867523e-05  -2.41209600e-05   9.59809549e-05
  -4.70681080e-05   9.31996256e-05   6.43376916e-05  -2.61820242e-05
   7.26967301e-06  -7.98481506e-06   1.40485063e-04  -9.43329651e-05
   1.98467005e-05  -5.33907514e-05  -1.22291647e-04   5.94479206e-05
   6.46285262e-05  -5.86524729e-05   6.44965185e-05   3.13518711e-07
   5.57013300e-05   1.88413498e-04   9.82349011e-05   7.65935783e-06
  -1.02457256e-04  -1.08079857e-05   4.01063939e-04  -1.13337808e-06
  -6.05598689e-05  -5.97166763e-05  -6.65440384e-05   3.51400813e-05
  -3.81440914e-05  -1.77962374e-05   2.15161594e-06   4.82048308e-05
   2.85141226e-07   2.08568949e-07  -6.18531412e-05   6.53392126e-05
